In [ ]:
import torch
from data_git import *
from os import path
from os import getcwd

In [ ]:
def create_hyper_graph(depth=5,dataset_path="muta/muta188",output="data"):
    bottom_clauses = {"pos":[],"neg":[]}
    sat = {"pos":[],"neg":[]}
    symbols = []
    pat = f"{getcwd()}/{output}/{dataset_path.split('/')[-1]}"
    if not path.exists(pat): makedirs(pat)
    # print(f"pat = {dataset_path}")
    size = 0
    for ppp in ["pos","neg"]:
        pat = f"{dataset_path}"
        pos_examples = load_examples(f"{pat}/{ppp}.pl")
        bk = f"{pat}/bk.pl"
        mode = f"{pat}/mode.pl"
        prolog = aleph_settings(mode, bk, depth, data_files={'train_pos': f"{pat}/{ppp}.pl"})
        for i in range(len(pos_examples)):
            prolog += [f':- sat({i+1}).']
        script = create_script(pat,prolog)
        prolog_output = run_aleph(script)
        with open(f"analise/{dataset_path.split('/')[-1]}_{ppp}_{depth}.txt","w") as f:
            f.write(prolog_output)
        bottom_clauses_raw = re.findall(r'\[bottom clause\]\n(.*?)\n\[literals\]', prolog_output,re.S)
        sats = re.findall(r'\[sat\] \[\d+\]\n\[(.*?)\]', prolog_output,re.S)
        print("-------------------------------")
        # continue
        for b,s in zip(bottom_clauses_raw,sats):
            clause = re.sub(r'[ \n]', '', b).split(':-')
            if len(clause) == 1:
                continue
            body = clause[1]
            variable = re.findall(r'([A-Z]=[- \d\.]+)[,\.]', body)
            var = {}
            for v in variable:
                v = v.split("=")
                var[v[0].strip()] = v[1].strip()
            for v in var:
                body = body.replace(f"{v},",f"{var[v]},").replace(f"{v})",f"{var[v]})")
            body = re.findall(r'(\w+\([\w,\.\-]+\))', body)
            size += len(body)
            for b in body:
                b = re.findall(r"\((.*?)\)",b)[0]
                for r in re.findall(r"([a-z_\d]+)",b):
                    symbols.append(r)

            bottom_clauses[ppp].append(sorted(body))
            sat[ppp].append(s)
        symbols = sorted(list(set(symbols)))
    modes = get_modes(f"{pat}/mode.pl")
    types = sorted(list(set([l for li in list(modes.values()) for l in li ])))
    predicates = sorted(list(set(list(modes.keys()))))
    x,y = create_subhgraphs(bottom_clauses, modes, symbols, predicates, types, sat)
    pat = f"{getcwd()}/{output}/{dataset_path.split('/')[-1]}"
    torch.save(y,f'{pat}/y{depth}h.pt')
    torch.save(x,f'{pat}/x{depth}h.pt')


In [ ]:
for d in ["muta/muta188","muta/muta42","carcinogenesis","alzheimers/choline","alzheimers/toxic","alzheimers/scopolamine","alzheimers/amine"]:
    create_hyper_graph(dataset_path=f"{getcwd()}/data_pre/{d}",output="output")